In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  


In [2]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

import keras

/home/rohita/rohit/spoof/work3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rohita/rohit/spoof/work3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rohita/rohit/spoof/work3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rohita/rohit/spoof/work3/lib/python3.5/site-packages

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import numpy as np
# import librosa as lb
from tensorflow import keras
import tensorflow as tf

from matplotlib import pyplot as plt
from keras.optimizers import Adam,SGD
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Lambda
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import AveragePooling1D
from keras.layers import BatchNormalization
from keras.layers.merge import add, concatenate
from keras.utils import plot_model
from keras.utils import to_categorical,Sequence
import pandas as pd
import math
import sincnet
# from tensorflow import set_random_seed
from keras import models, layers
import numpy as np
import sincnet
from keras.layers import Dense, Dropout, Activation
from keras.layers import MaxPooling1D,MaxPooling2D, Conv1D, LeakyReLU, BatchNormalization, Dense, Flatten
from keras.layers import InputLayer, Input
from keras.models import Model
# from tensorflow.python.keras.utils.data_utils import Sequence
from sklearn.preprocessing import StandardScaler
import pywt
from sklearn.decomposition import PCA
import sys
import tensorflow.contrib.slim as slim
import pickle
import os

In [4]:
def generator(data, labels, batch_size):
    """
    Yields the next training batch.
    Suppose `samples` is an array [[audio1,label1], [audio2,label2],...].
    """
    total_batches = int(data.shape[0]/batch_size)
    for i in range(total_batches):
        X_train,y_train = create_batches(data, labels, batch_size,i)
        yield X_train, y_train

In [5]:
def create_batches(data,labels,batch_size,batch_number):
    wlen = 3200
    fact_amp = 0.2
    # Initialization of the minibatch (batch_size,[0=>x_t,1=>x_t+N,1=>random_samp])
    sig_batch=np.zeros([batch_size,wlen])
    lab_batch=[]
    if batch_number == 0:
        signal_id_arr=np.array([s for s in range(0,batch_size)])
    else:
        offset = (batch_number*batch_size) 
        signal_id_arr=np.array([s for s in range(offset,offset+batch_size)])
    rand_amp_arr = np.random.uniform(1.0-fact_amp,1+fact_amp,batch_size)
    for i in range(batch_size): 
#         print(i)
        signal = data[signal_id_arr[i]]
        # accesing to a random chunk
        signal_len=signal.shape[0]
        signal_beg=np.random.randint(signal_len-wlen-1) #randint(0, snt_len-2*wlen-1)
        signal_end=signal_beg+wlen
        sig_batch[i,:]=signal[signal_beg:signal_end]*rand_amp_arr[i]
        y=labels.iloc[signal_id_arr[i],-1]
        lab_batch.append(y)
    a, b = np.shape(sig_batch)
    sig_batch = sig_batch.reshape((a, b, 1))
    return sig_batch, to_categorical(np.array(lab_batch),num_classes = 2)

In [6]:
def res_conv_block(X,in_channels,out_channels,stage,block,dilation=1):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    X_shortcut = X
    
    X = BatchNormalization(name=bn_name_base+'a')(X)
    X = Activation('relu')(X)
    X = Conv1D(in_channels, 3, padding='valid',use_bias = False, name= conv_name_base+'a')(X)
    X = BatchNormalization(name=bn_name_base+'b')(X)
    X = Activation('relu')(X)
    X = Conv1D(in_channels, 3, padding='valid',use_bias = False, name= conv_name_base+'b')(X)
    print(X.shape)
    paddings = tf.constant([[0, 0],   # the batch size dimension
                          [2, 2],   # top and bottom of image
                          [0, 0]])  # the channels dimension
    X = Lambda(lambda x: tf.pad(x, paddings, mode='CONSTANT',
                        constant_values=0.0))(X)
    X = concatenate([X , X_shortcut])
    X = BatchNormalization(name = bn_name_base+'c')(X)
    X = Activation('relu')(X)
    X = Conv1D(out_channels, 3, padding='valid',use_bias = False, dilation_rate = dilation, name = conv_name_base+'c')(X)

    return X

In [6]:
    
input_shape = None,3200,1


inputs = tf.placeholder(tf.float32, shape=input_shape, name= 'the_input')
learning_rate = tf.placeholder(tf.float32, shape=[])

sinc = sincnet.SincConv1D(64, 251, 16000)(inputs)
sinc_pool = MaxPooling1D(pool_size=3,name = 'sinc_pool')(sinc)
sinc_layer_norm = sincnet.LayerNorm(name = 'sinc_layer_norm')(sinc_pool)
sinc_relu = LeakyReLU(alpha=0.2, name = 'sinc_relu')(sinc_layer_norm)

sinc_conv = Conv1D(64, 5, strides=2, padding='valid')(sinc_relu)
sinc_pool_1 = MaxPooling1D(pool_size=3,name = 'sinc_pool_1')(sinc_conv)
sinc_layer_norm_1 = sincnet.LayerNorm(name = 'sinc_layer_norm_1')(sinc_pool_1)
sinc_relu_1 = LeakyReLU(alpha=0.2, name = 'sinc_relu_1')(sinc_layer_norm_1)
 

sinc_conv_1 = Conv1D(64, 5, strides=2, padding='valid')(sinc_relu_1)
sinc_pool_2 = MaxPooling1D(pool_size=3,name = 'sinc_pool_1')(sinc_conv_1)

sinc_layer_norm_2= sincnet.LayerNorm(name = 'sinc_layer_norm_1')(sinc_pool_2)
sinc_relu_2= LeakyReLU(alpha=0.2, name = 'sinc_relu_1')(sinc_layer_norm_2)
flat = Flatten()(sinc_relu_2)
#concate level one and level two decomposition
# concate_level_2 = concatenate([relu_1_2,sinc_relu_1])
# print(concate_level_2.shape)
# res_conv_1 = res_conv_block(sinc_relu_1, 128, 16, 1, 'a', 4)
# res_conv_2 = res_conv_block(res_conv_1, 16, 8, 2, 'a', 8)
# res_conv_3 = res_conv_block(res_conv_2, 8, 4, 3, 'a', 16)
# res_conv_4 = res_conv_block(res_conv_3, 4, 2, 4, 'a', 32)
# res_conv_5 = res_conv_block(res_conv_4, 2, 1, 5, 'a', 64)

# res_norm = BatchNormalization(name='res_norm')(res_conv_5)
# res_relu = Activation('relu')(res_norm)


# pool_5_1 = AveragePooling1D(pool_size=3, padding='same', name='avg_pool_5_1')(sinc_relu_2)
# flat_5_1 = Flatten(name='flat_5_1')(pool_5_1) 

fc_5 = Dense(2048, name='fc_5')(flat)
norm_5 = BatchNormalization(momentum=0.05, epsilon=1e-5)(fc_5)
relu_5 =  LeakyReLU(alpha=0.2)(norm_5)
drop_5 = Dropout(0.5, name='drop_5')(relu_5)

fc_6 = Dense(2048, name='fc_6')(drop_5)
norm_6 = BatchNormalization(momentum=0.05, epsilon=1e-5)(fc_6)
relu_6 =  LeakyReLU(alpha=0.2)(norm_6)
drop_6 = Dropout(0.5, name='drop_6')(relu_6)

fc_7= Dense(2048, name='fc_6')(drop_6)
norm_7= BatchNormalization(momentum=0.05, epsilon=1e-5)(fc_7)
relu_7=  LeakyReLU(alpha=0.2)(norm_7)
drop_7= Dropout(0.5, name='drop_6')(relu_7)                                                        

                                                        
output = Dense(2, activation=tf.nn.softmax)(drop_7)


    

In [12]:
model = create_model()

In [7]:
model_path = '/home/rohita/rohit/spoof/npy_data_asvspoof/Sincnet/equal_human_spoof'
saver = tf.train.Saver()
# save_path = saver.save(sess, model_path)
with sess.as_default():
    latest = tf.train.latest_checkpoint(model_path)

In [8]:
latest


'/home/rohita/rohit/spoof/npy_data_asvspoof/Sincnet/equal_human_spoof/model.ckpt'

In [9]:
X = np.load("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev.npy",allow_pickle=True)
with open("/home/rohita/rohit/spoof/npy_data_asvspoof/ASVspoof2019_dev_df.pkl", 'rb') as pickle_file:
    y = pickle.load(pickle_file)

X= np.array([s[0] for s in X])

In [10]:
X.shape

(24843,)

In [12]:
init_op = tf.global_variables_initializer()
batch_size = 1
scores = []
total_batch = int(X.shape[0]/batch_size)
gen = generator(X, y, batch_size)
with sess.as_default():
    sess.run(init_op)
    for i in range(total_batch):
        X_batch, y_batch = next(gen)
        if i%100==0 | i ==0:
            print(i)
        score = (sess.run(output,feed_dict={inputs: X_batch, tf.keras.backend.learning_phase(): 0}))
#         print(score.shape)
        scores.append(score)
            

0


In [13]:
scores = np.array(scores)
scores = scores.reshape(-1,2)

In [14]:
y['scores_1'] = scores[:,0]

In [15]:
y['scores_2'] = scores[:,1]

In [17]:
y

,speaker_id,file_id,system_id,label,scores_1,scores_2
0,LA_0071,LA_D_9616032,A05,1,0.648249,0.351751
1,LA_0071,LA_D_4264656,A02,1,0.633547,0.366453
2,LA_0073,LA_D_6273410,A04,1,0.475878,0.524122
3,LA_0074,LA_D_6947992,A05,1,0.524474,0.475526
4,LA_0072,LA_D_9410329,A06,1,0.533137,0.466863
5,LA_0077,LA_D_3624723,A05,1,0.638721,0.361279
6,LA_0070,LA_D_2591082,A02,1,0.613276,0.386724
7,LA_0075,LA_D_5223993,A03,1,0.518717,0.481283
8,LA_0102,LA_D_7153183,-,0,0.479007,0.520993
9,LA_0072,LA_D_3428449,A01,1,0.504549,0.495451


In [20]:
file1 = open("/home/rohita/rohit/spoof/npy_data_asvspoof/dev_scores.txt","w")

for i in range(y.shape[0]):
    if y.iloc[i,2] == '-':
        L = y.iloc[i,1] + " " + y.iloc[i,2] + " bonafide" + " " + str(y.iloc[i,-2] - y.iloc[i,-1]) + '\n'
    else:
        L = y.iloc[i,1] + " " + y.iloc[i,2] + " spoof" + " " + str(y.iloc[i,-2] - y.iloc[i,-1]) + '\n'
    file1.writelines(L) 
file1.close()

In [5]:
unique, counts = np.unique(y_train.iloc[:,3], return_counts=True)
print(dict(zip(unique, counts)))

{0: 2579, 1: 3800}


In [6]:
y_train.iloc[1888,:]#,y_train.iloc[2176,:],y_train.iloc[23213,:]

speaker_id         LA_0088
file_id       LA_T_1241016
system_id                0
label                    0
Name: 9345, dtype: object

In [21]:
y_train = y_train.reset_index(drop=True)

In [27]:
print(y_train.iloc[12754])

speaker_id         LA_0086
file_id       LA_T_9513704
system_id                0
label                    0
Name: 10549, dtype: object


In [29]:
y_train.loc[y_train.iloc[:,1]=='LA_T_9513704'].index

Int64Index([10549], dtype='int64')

In [30]:
y_train.iloc[10549,:]

speaker_id         LA_0096
file_id       LA_T_5159226
system_id                5
label                    1
Name: 10792, dtype: object

In [23]:
for s in range(0,128):
    print(s)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
